In [1]:
import numpy as np
import math

t = 0  # number of this round
t_x = [] #matrix of times
T = 150000  # numbers of rounds
l = 30  # ||a|| <= L
d = 3 #dimension of arm
m = 3 #||theta|| < m
theta_stimato = [[]] #np.array([])  theta
A = [] #collector of arm
A_y = [] #collector of arm
num_arms = 0 #number of arms
num_arms_y = 0 #number of arms y
num_theta = 0 #num theta
num_M = 0 #num world
M = [] #collector of matrix for each world
Th = [] #collector of theta
W = [[]] #distribution of probability to play arms
W_y = [] #distribution of probability to play arms
N = [[]] #number of times an arm is played
N_y = [] #number of times an arm is played
Y = [[]] #collection of reward
Y_y = [] #collection of reward
H = [[]] #collection of hedge losses
H_y = [] #collection of hedge losses
L = [[]] #collection of losses
L_y = [] #collection of losses
Mixability_Gap = [[]] #collection ofa all mixability gap at each t
Mixability_Gap_y = [] #collection ofa all mixability gap at each t
AI = [[]] #number of the arm played at time I
AI_y = [] #number of the arm played at time I
nu = 0.01 #regularization paramether
nu_y = 0.01 #regularization paramether
eta = [] #learning paramether
eta_y = 0.1 #learning paramether
M_vero = 0
curr_answ = 0
curr_y = -1

def normalize(v):
    sum = np.sum(v)
    if sum > 0:
        return v / sum
    else:
        return np.array([0.33]*len(v))
    
class theta():
    vector = np.array([])
    best_arm = -1
    id
    num_m = -1
    num_y = -1

    def __init__(self, param, numm, numy):
        self.vector = param
        global Th, num_theta
        self.id = num_theta
        Th.append(self)
        num_theta +=1
        self.num_m = numm
        self.num_y = numy
        #i = 0
        #best = -5
        #while i < num_arms:
            #tmp = np.dot( A[i].arm_vector, np.transpose(self.vector))
            #if best < tmp:
                #best = tmp
                #self.best_arm = i

class arm():
    arm_vector = np.array([])
    id = 0
    best_theta = -1

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms, A
        A.append(self)
        self.id = num_arms
        num_arms +=1
    
    def best(self):
        max = -5
        for i in range (0, num_theta):
            tmp = np.matmul(self.arm_vector, Th[i].vector)
            if tmp > max:
                self.best_theta = i
                max = tmp
                            

    def pull_arm(self):
        y = np.dot(self.arm_vector, np.transpose(np.dot(M[M_vero].matrix, np.transpose(A_y[curr_y].arm_vector)))) + np.random.normal(0, 1)
        Y[curr_y].append(y)
        N[curr_y][self.id] += 1
        AI[curr_y].append(self.id)
        A_y[curr_y].pull_arm(y)
        return y
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)

def design_matrix(w):
    V = []
    i = 0
    while i < num_arms:
        b = [A[i].arm_vector]
        a = np.swapaxes(b, 0, 1)
        ris = np.matmul(a, b) * w[i]
        if (i == 0):
            V = ris
        else:
            V += ris
        i+=1
    return V

def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

def arm_to_play(y_index):
    diff = np.array(N[y_index]) - np.array(W[y_index])
    i = 0
    arm = 0
    min = +5000
    while i < num_arms:
        if diff[i] < min:
            min = diff[i]
            arm = i
        i += 1
    return arm

def stima_theta(y_index):
    V = design_matrix(N[y_index])
    I = np.identity(d)
    p = np.linalg.inv(V + nu*I)
    sum = 0
    print("stima_theta")
    if t_x[curr_y] > 1:
        for i in range (0, t_x[y_index]):
            print("y_index", y_index)
            print("t_x", t_x)
            print("t_x y_index", t_x[y_index])
            print("Y[y_index]", Y[y_index])
            print("A", A)
            print("AI[y_index]", AI[y_index])
            print("AI", AI)
            print("A[AI[y_index][i]].arm_vector", A[AI[y_index][i]].arm_vector)
            sum += Y[y_index][i] * np.array(A[AI[y_index][i]].arm_vector)
        res = np.matmul(p, sum)
        global theta_stimato
        theta_stimato[curr_y] = res
    else:
        res = [0]*num_arms
        theta_stimato[curr_y] = res
    print("theta stimato", theta_stimato)
    return res

def best_answer():
    max = -100
    best = -1
    for i in range (0, num_arms):
        curr = np.matmul(A[i].arm_vector, theta_stimato[curr_y])
        if curr > max:
            max = curr
            best = i
    global curr_answ
    curr_answ = best
    return best

def best_arm():
    max = -100
    best = -1
    for i in range (0, num_arms):
        curr = np.matmul(A[i].arm_vector, Th[theta_vero].vector)
        if curr > max:
            max = curr
            best = i
    global curr_answ
    curr_answ = best
    return best


def beta(time, delta):
    n = np.abs(np.random.normal(0, 1))
    res = math.sqrt(math.log10(1/delta) + d/2*math.log10(1 + (time*(l**2))/(n*d))) + math.sqrt(n/2)*m
    return res

def best_response():
    min = 1000000
    theta_adv = -11
    print("num_theta", num_theta)
    for i in range (0, num_theta):
        print("curr_answ", curr_answ)
        if not i == A[curr_answ].best_theta:
            print("fattori curr inizio")
            print("curr_y", curr_y)
            print("np.array(theta_stimato[curr_y])", np.array(theta_stimato[curr_y]))
            print("np.array(Th[i].vector)", np.array(Th[i].vector))
            print("W", W)
            print("design_matrix(W[curr_y])", design_matrix(W[curr_y]))
            print("norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y]))", norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])))
            print("fattori curr fine")
            curr = (norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])))**2
            print("curr", curr)
            if curr < min:
                min = curr
                theta_adv = i
    print("theta_adv", theta_adv)
    return theta_adv

def calcola_U (id_arm):
    xi = [0]*d
    q2 = 4*(l**2)*(m**2)*100
    q1 = 0
    s1 = np.array(theta_stimato[curr_y])
    print("best response", best_response())
    s2 = np.array(Th[best_response()].vector)
    s3 = np.array(A[id_arm].arm_vector)
    p1 = np.matmul((np.array(theta_stimato[curr_y]) - np.array(Th[best_response()].vector)), np.array(A[id_arm].arm_vector))
    p2 = math.sqrt(2*beta(T, 1/T**3))*norm_2_mat(np.array(A[id_arm].arm_vector), design_matrix(normalize(N[curr_y])) + np.random.normal(0, 1)*np.identity(d))
    r1 = (p1 + p2)**2
    r2 = (p1 - p2)**2
    if r1> r2:
        q1 = r1
    else:
        q1 = r2
    if q1 > q2:
        print("q2")
        return q2
    else:
        print("q1")
        return q1

def calcola_loss():
    loss = 0
    arm_play = arm_to_play(curr_y)
    best_res = best_response ()
    for i in range (0, num_arms):
        loss += W[curr_y][i]*calcola_U(i)/2
    L[curr_y][arm_play] += (4*(l**2)*(m**2) - loss)/1000000
    print("    LOSS", loss/10000)
    print("    INV LOSS", (4*(l**2)*(m**2) - loss)/1000000)
    L_y[curr_y] += loss/10000
    return (4*(l**2)*(m**2) - loss)/1000000

In [2]:
def h():
    return np.dot(W[curr_y], L[curr_y])

def mix_loss():
    sum = 0
    print("mix_loss computation")
    for i in range (0, num_arms):
        print("L[curr_y][i]", L[curr_y][i])
        print("-eta[curr_y]", -eta[curr_y])
        print("math.exp(-eta[curr_y] * L[curr_y][i])", math.exp(-eta[curr_y] * L[curr_y][i]))
        print("W[curr_y][i]", W[curr_y][i])
        sum += W[curr_y][i] * math.exp(-eta[curr_y] * L[curr_y][i])
    print("sum", sum)
    return -1/eta[curr_y] * np.log(sum)

def mixability_gap():
    print("mixability gap")
    print("h", h())
    print("mix_loss", mix_loss())
    m_g = h() - mix_loss()
    Mixability_Gap[curr_y].append(m_g)

def new_eta():
    print("   NEW ETA!!")
    if t_x[curr_y]>1:
        sum = 0
        print("t_x curry", t_x[curr_y])
        for i in range (0, t_x[curr_y]):
            print("mix gap", Mixability_Gap[curr_y][i])
            sum += Mixability_Gap[curr_y][i]
        print("sum eta", sum)
        eta[curr_y] = math.log(d)/sum
        
def generate_w():
    sum=0
    global W
    for i in range (0, num_arms):
        print("      GENERATE W")
        print("curr y", curr_y)
        print("eta_curr_y", eta[curr_y])
        print("eta", eta)
        print("L curr y", L[curr_y])
        print("num_arms", num_arms)
        W[curr_y][i] = math.exp(-eta[curr_y] * L[curr_y][i])/num_arms
        sum += math.exp(-eta[curr_y] * L[curr_y][i])/num_arms
        print("sum", sum)
    print("W", W)
    for i in range (0, num_arms):
        W[curr_y][i] = W[curr_y][i]/sum
    return W

In [3]:
class arm_y():
    arm_vector = np.array([])
    id = 0
    best_theta = -1

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms_y, A_y
        self.id = num_arms_y
        num_arms_y +=1
        A_y.append(self)
        W_y.append(0)
        N_y.append(0)
        L_y.append(0)
        Mixability_Gap.append([])
        global Y, H, L, W, N, theta_stimato, AI
        L = [[0] * num_arms] * num_arms_y
        W = [[0] * num_arms] * num_arms_y
        N = [[0] * num_arms] * num_arms_y
        theta_stimato = [[0] * num_arms] * num_arms_y
        Y = [[0]] * num_arms_y
        AI = [[0]] * num_arms_y
        H = [[0]] * num_arms_y
        t_x.append(0)
        eta.append(0.1)
        
    def pull_arm(reward, self):
        Y_y.append(reward)
        N_y[curr_y] += 1
        AI_y.append(curr_y)
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)

def h_y():
    return np.dot(W_y, L_y)

def mix_loss_y():
    sum = 0
    for i in range (0, num_arms_y):
        sum += W_y[i] * math.exp(-eta_y * L_y[i])
    return -1/eta_y * np.log(sum)

def mixability_gap_y():
    m_g = h_y() - mix_loss_y()
    Mixability_Gap_y.append(m_g)

def new_eta_y():
    if t>0:
        sum = 0
        for i in range (0, t):
            sum += Mixability_Gap_y[i]
        eta_y = math.log(d)/sum
        
def generate_w_y():
    sum=0
    for i in range (0, num_arms_y):
        W_y[i] = math.exp(-eta_y * L_y[i])/num_arms_y
        sum += math.exp(-eta_y * L_y[i])/num_arms_y
    for i in range (0, num_arms_y):
        W_y[i] = W_y[i]/sum
    print("W y", W_y)
    return W_y

def arm_to_play_y():
    diff = np.array(N_y) - np.array(W_y)
    i = 0
    arm = 0
    min = +5000
    while i < num_arms_y:
        if diff[i] < min:
            min = diff[i]
            arm = i
        i += 1
    global curr_y
    curr_y = arm
    return arm

class world():
    matrix = np.array([])
    id

    def __init__(self, param):
        self.matrix = param
        global M, num_M
        self.id = num_M
        M.append(self)
        num_M +=1
        
def generate_theta():
    for i in range (0, num_M):
        for j in range (0, num_arms_y):
            theta(np.dot(M[i].matrix, np.transpose(A_y[j].arm_vector)), i, j)

In [4]:
a = arm([1.0,2.0,3.0])
b = arm([2.0,5.0,1.0])
c = arm([1.0, 3.0, 1.0])
g = arm_y([3,0,2])
f = arm_y([1,1,1])
w = [0.34, 0.33, 0.33]
V = design_matrix(w)
s = world([[0.1, 0.4, 0.5],[0.2, 0.4, 0.4],[0.1, 0.5, 0.4]])
p = world([[0.4, 0.6, 0.2],[0.4, 0.5, 0.1],[0.1, 0.7, 0.2]])
q = world([[0.01, 0.1, 0.8], [0.01, 0.1, 0.8], [0.01, 0.1, 0.8]])
generate_theta()
for i in range(0, num_arms):
    A[i].best()
for i in range(0, num_arms_y*num_M):
    print(Th[i].vector)


[1.3 1.4 1.1]
[1. 1. 1.]
[1.6 1.4 0.7]
[1.2 1.  1. ]
[1.63 1.63 1.63]
[0.91 0.91 0.91]


In [5]:
print("theta_stimato", theta_stimato)
print("A", A)
print("A_y", A_y)
print("num_arms", num_arms)
print("num_arms_y", num_arms_y)
print("num_theta", num_theta)
print("num_M", num_M)
print("M", M)
print("Th", Th)
print("W", W)
print("W_y", W_y)
print("N", N)
print("N_y", N_y)
print("Y", Y)
print("Y_y", Y_y)
print("H", H)
print("H_y", H_y)
print("L", L)
print("L_y", L_y)
print("Mixability_Gap", Mixability_Gap)
print("Mixability_Gap_y", Mixability_Gap_y)
print("AI", AI)
print("AI_y", AI_y)
print("nu", nu)
print("nu_y", nu_y)
print("eta", eta)
print("eta_y", eta_y)
print("M_vero", M_vero)
print("curr_answ", curr_answ)
print("curr_y", curr_y)

theta_stimato [[0, 0, 0], [0, 0, 0]]
A [<__main__.arm object at 0x000001C5F0DCF3C8>, <__main__.arm object at 0x000001C5F0DCF548>, <__main__.arm object at 0x000001C5F0DCF648>]
A_y [<__main__.arm_y object at 0x000001C5F0DCF2C8>, <__main__.arm_y object at 0x000001C5F0DCF608>]
num_arms 3
num_arms_y 2
num_theta 6
num_M 3
M [<__main__.world object at 0x000001C5F0DCF348>, <__main__.world object at 0x000001C5F0C1EF88>, <__main__.world object at 0x000001C5F0C1E548>]
Th [<__main__.theta object at 0x000001C5F0E3F588>, <__main__.theta object at 0x000001C5F0DC5648>, <__main__.theta object at 0x000001C5F0E3F948>, <__main__.theta object at 0x000001C5F0E3FA88>, <__main__.theta object at 0x000001C5F0E3F9C8>, <__main__.theta object at 0x000001C5F0E3FB48>]
W [[0, 0, 0], [0, 0, 0]]
W_y [0, 0]
N [[0, 0, 0], [0, 0, 0]]
N_y [0, 0]
Y [[0], [0]]
Y_y []
H [[0], [0]]
H_y []
L [[0, 0, 0], [0, 0, 0]]
L_y [0, 0]
Mixability_Gap [[], [], []]
Mixability_Gap_y []
AI [[0], [0]]
AI_y []
nu 0.01
nu_y 0.01
eta [0.1, 0.1]
e

In [6]:
t += 1
mixability_gap_y()
new_eta_y()
generate_w_y()
curr_y = arm_to_play_y()
t_x[curr_y] +=1
stima_theta(curr_y)
best_answer()
curr = arm_to_play(curr_y)
mixability_gap()
new_eta()
generate_w()
print("curr y", curr_y)
err = calcola_loss()
print("err", err)
A[curr].pull_arm()

W y [0.5, 0.5]
stima_theta
theta stimato [[0, 0, 0], [0, 0, 0]]
mixability gap
h 0
mix_loss computation
L[curr_y][i] 0
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 1.0
W[curr_y][i] 0
L[curr_y][i] 0
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 1.0
W[curr_y][i] 0
L[curr_y][i] 0
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 1.0
W[curr_y][i] 0
sum 0.0
mix_loss inf
mix_loss computation
L[curr_y][i] 0
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 1.0
W[curr_y][i] 0
L[curr_y][i] 0
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 1.0
W[curr_y][i] 0
L[curr_y][i] 0
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 1.0
W[curr_y][i] 0
sum 0.0
   NEW ETA!!
      GENERATE W
curr y 0
eta_curr_y 0.1
eta [0.1, 0.1]
L curr y [0, 0, 0]
num_arms 3
sum 0.3333333333333333
      GENERATE W
curr y 0
eta_curr_y 0.1
eta [0.1, 0.1]
L curr y [0, 0, 0]
num_arms 3
sum 0.6666666666666666
      GENERATE W
curr y 0
eta_curr_y 0.1
eta [0.1, 0.1]
L curr y [0, 0, 0]


C:\Users\edoar\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in log
C:\Users\edoar\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  



np.array(Th[i].vector) [0.91 0.91 0.91]
W [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
design_matrix(W[curr_y]) [[ 2.          5.          2.        ]
 [ 5.         12.66666667  4.66666667]
 [ 2.          4.66666667  3.66666667]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 5.874024741747915
fattori curr fine
curr 34.50416666666666
theta_adv 5
q1
num_theta 6
curr_answ 0
fattori curr inizio
curr_y 0
np.array(theta_stimato[curr_y]) [0 0 0]
np.array(Th[i].vector) [1.3 1.4 1.1]
W [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
design_matrix(W[curr_y]) [[ 2.          5.          2.        ]
 [ 5.         12.66666667  4.66666667]
 [ 2.          4.66666667  3.66666667]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 8.422390792801453
fattori

np.array(Th[i].vector) [0.91 0.91 0.91]
W [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
design_matrix(W[curr_y]) [[ 2.          5.          2.        ]
 [ 5.         12.66666667  4.66666667]
 [ 2.          4.66666667  3.66666667]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 5.874024741747915
fattori curr fine
curr 34.50416666666666
theta_adv 5
num_theta 6
curr_answ 0
fattori curr inizio
curr_y 0
np.array(theta_stimato[curr_y]) [0 0 0]
np.array(Th[i].vector) [1.3 1.4 1.1]
W [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
design_matrix(W[curr_y]) [[ 2.          5.          2.        ]
 [ 5.         12.66666667  4.66666667]
 [ 2.          4.66666667  3.66666667]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 8.422390792801453
fattori cur

6.386056580331619

In [7]:
print("theta_stimato", theta_stimato)
print("A", A)
print("A_y", A_y)
print("num_arms", num_arms)
print("num_arms_y", num_arms_y)
print("num_theta", num_theta)
print("num_M", num_M)
print("M", M)
print("Th", Th)
print("W", W)
print("W_y", W_y)
print("N", N)
print("N_y", N_y)
print("Y", Y)
print("Y_y", Y_y)
print("H", H)
print("H_y", H_y)
print("L", L)
print("L_y", L_y)
print("Mixability_Gap", Mixability_Gap)
print("Mixability_Gap_y", Mixability_Gap_y)
print("AI", AI)
print("AI_y", AI_y)
print("nu", nu)
print("nu_y", nu_y)
print("eta", eta)
print("eta_y", eta_y)
print("M_vero", M_vero)
print("curr_answ", curr_answ)
print("curr_y", curr_y)

theta_stimato [[0, 0, 0], [0, 0, 0]]
A [<__main__.arm object at 0x000001C5F0DCF3C8>, <__main__.arm object at 0x000001C5F0DCF548>, <__main__.arm object at 0x000001C5F0DCF648>]
A_y [<__main__.arm_y object at 0x000001C5F0DCF2C8>, <__main__.arm_y object at 0x000001C5F0DCF608>]
num_arms 3
num_arms_y 2
num_theta 6
num_M 3
M [<__main__.world object at 0x000001C5F0DCF348>, <__main__.world object at 0x000001C5F0C1EF88>, <__main__.world object at 0x000001C5F0C1E548>]
Th [<__main__.theta object at 0x000001C5F0E3F588>, <__main__.theta object at 0x000001C5F0DC5648>, <__main__.theta object at 0x000001C5F0E3F948>, <__main__.theta object at 0x000001C5F0E3FA88>, <__main__.theta object at 0x000001C5F0E3F9C8>, <__main__.theta object at 0x000001C5F0E3FB48>]
W [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
W_y [0.5, 0.5]
N [[1, 0, 0], [1, 0, 0]]
N_y [1, 0]
Y [[0, 6.386056580331619], [0, 6.386056580331619]]
Y_y [<__main__.arm_y ob

In [8]:
t += 1
mixability_gap_y()
new_eta_y()
generate_w_y()
curr_y = arm_to_play_y()
t_x[curr_y] +=1
stima_theta(curr_y)
best_answer()
curr = arm_to_play(curr_y)
mixability_gap()
new_eta()
generate_w()
err = calcola_loss()
print(err)
A[curr].pull_arm()

W y [0.49514518203059904, 0.504854817969401]
stima_theta
theta stimato [[0, 0, 0], [0, 0, 0]]
mixability gap
h 0.010152670594137126
mix_loss computation
L[curr_y][i] 0.03045801178241138
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 0.9969588325684833
W[curr_y][i] 0.3333333333333333
L[curr_y][i] 0
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 1.0
W[curr_y][i] 0.3333333333333333
L[curr_y][i] 0
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 1.0
W[curr_y][i] 0.3333333333333333
sum 0.9989862775228278
mix_loss 0.010142366413118031
mix_loss computation
L[curr_y][i] 0.03045801178241138
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 0.9969588325684833
W[curr_y][i] 0.3333333333333333
L[curr_y][i] 0
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 1.0
W[curr_y][i] 0.3333333333333333
L[curr_y][i] 0
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 1.0
W[curr_y][i] 0.3333333333333333
sum 0.9989862775228278
   NEW ETA!!
      GENERATE W
curr y 1
eta

fattori curr inizio
curr_y 1
np.array(theta_stimato[curr_y]) [0 0 0]
np.array(Th[i].vector) [1. 1. 1.]
W [[0.33265683256451667, 0.33367158371774164, 0.33367158371774164], [0.33265683256451667, 0.33367158371774164, 0.33367158371774164]]
design_matrix(W[curr_y]) [[ 2.00101475  5.00304425  1.99898525]
 [ 5.00304425 12.67546118  4.66531367]
 [ 1.99898525  4.66531367  3.66125466]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 6.455417641268581
fattori curr fine
curr 41.67241692320161
curr_answ 0
fattori curr inizio
curr_y 1
np.array(theta_stimato[curr_y]) [0 0 0]
np.array(Th[i].vector) [1.6 1.4 0.7]
W [[0.33265683256451667, 0.33367158371774164, 0.33367158371774164], [0.33265683256451667, 0.33367158371774164, 0.33367158371774164]]
design_matrix(W[curr_y]) [[ 2.00101475  5.00304425  1.99898525]
 [ 5.00304425 12.67546118  4.66531367]
 [ 1.99898525  4.66531367  3.66125466]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), desi

np.array(Th[i].vector) [1.2 1.  1. ]
W [[0.33265683256451667, 0.33367158371774164, 0.33367158371774164], [0.33265683256451667, 0.33367158371774164, 0.33367158371774164]]
design_matrix(W[curr_y]) [[ 2.00101475  5.00304425  1.99898525]
 [ 5.00304425 12.67546118  4.66531367]
 [ 1.99898525  4.66531367  3.66125466]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 6.734513732603981
fattori curr fine
curr 45.3536752146316
curr_answ 0
curr_answ 0
fattori curr inizio
curr_y 1
np.array(theta_stimato[curr_y]) [0 0 0]
np.array(Th[i].vector) [0.91 0.91 0.91]
W [[0.33265683256451667, 0.33367158371774164, 0.33367158371774164], [0.33265683256451667, 0.33367158371774164, 0.33367158371774164]]
design_matrix(W[curr_y]) [[ 2.00101475  5.00304425  1.99898525]
 [ 5.00304425 12.67546118  4.66531367]
 [ 1.99898525  4.66531367  3.66125466]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 5.874430053554408
fattori curr

8.197770099053855

In [9]:
N

[[1, 1, 0], [1, 1, 0]]

In [10]:
for i in range (100):
    print("    INIZIO!!!")
    t += 1
    mixability_gap_y()
    new_eta_y()
    generate_w_y()
    curr_y = arm_to_play_y()
    print("curry", curr_y)
    t_x[curr_y] +=1
    stima_theta(curr_y)
    best_answer()
    curr = arm_to_play(curr_y)
    mixability_gap()
    new_eta()
    generate_w()
    err = calcola_loss()
    print(err)
    A[curr].pull_arm()

    INIZIO!!!
W y [0.4985621181243758, 0.5014378818756242]
curry 1
stima_theta
y_index 1
t_x [1, 2]
t_x y_index 2
Y[y_index] [0, 6.386056580331619, 8.197770099053855]
A [<__main__.arm object at 0x000001C5F0DCF3C8>, <__main__.arm object at 0x000001C5F0DCF548>, <__main__.arm object at 0x000001C5F0DCF648>]
AI[y_index] [0, 0, 1]
AI [[0, 0, 1], [0, 0, 1]]
A[AI[y_index][i]].arm_vector [1.0, 2.0, 3.0]
y_index 1
t_x [1, 2]
t_x y_index 2
Y[y_index] [0, 6.386056580331619, 8.197770099053855]
A [<__main__.arm object at 0x000001C5F0DCF3C8>, <__main__.arm object at 0x000001C5F0DCF548>, <__main__.arm object at 0x000001C5F0DCF648>]
AI[y_index] [0, 0, 1]
AI [[0, 0, 1], [0, 0, 1]]
A[AI[y_index][i]].arm_vector [1.0, 2.0, 3.0]
theta stimato [[0, 0, 0], array([ 3.26752625e-04, -4.89475433e-01,  2.45162495e+00])]
mixability gap
h 0.020486951412177455
mix_loss computation
L[curr_y][i] 0.03045801178241138
-eta[curr_y] -0.1
math.exp(-eta[curr_y] * L[curr_y][i]) 0.9969588325684833
W[curr_y][i] 0.332656832564516

W [[0.0, 0.0, 1.0], [0.0, 0.0, 1.0]]
design_matrix(W[curr_y]) [[1. 3. 1.]
 [3. 9. 3.]
 [1. 3. 1.]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 5.516474597239768
fattori curr fine
curr 30.431491981991666
curr_answ 0
fattori curr inizio
curr_y 1
np.array(theta_stimato[curr_y]) [ 3.26752625e-04 -4.89475433e-01  2.45162495e+00]
np.array(Th[i].vector) [1.2 1.  1. ]
W [[0.0, 0.0, 1.0], [0.0, 0.0, 1.0]]
design_matrix(W[curr_y]) [[1. 3. 1.]
 [3. 9. 3.]
 [1. 3. 1.]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 4.216474597239768
fattori curr fine
curr 17.778658029168263
curr_answ 0
curr_answ 0
fattori curr inizio
curr_y 1
np.array(theta_stimato[curr_y]) [ 3.26752625e-04 -4.89475433e-01  2.45162495e+00]
np.array(Th[i].vector) [0.91 0.91 0.91]
W [[0.0, 0.0, 1.0], [0.0, 0.0, 1.0]]
design_matrix(W[curr_y]) [[1. 3. 1.]
 [3. 9. 3.]
 [1. 3. 1.]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i

np.array(theta_stimato[curr_y]) [ 3.26752625e-04 -4.89475433e-01  2.45162495e+00]
np.array(Th[i].vector) [1.6 1.4 0.7]
W [[0.0, 0.0, 1.0], [0.0, 0.0, 1.0]]
design_matrix(W[curr_y]) [[1. 3. 1.]
 [3. 9. 3.]
 [1. 3. 1.]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 5.516474597239768
fattori curr fine
curr 30.431491981991666
curr_answ 0
fattori curr inizio
curr_y 1
np.array(theta_stimato[curr_y]) [ 3.26752625e-04 -4.89475433e-01  2.45162495e+00]
np.array(Th[i].vector) [1.2 1.  1. ]
W [[0.0, 0.0, 1.0], [0.0, 0.0, 1.0]]
design_matrix(W[curr_y]) [[1. 3. 1.]
 [3. 9. 3.]
 [1. 3. 1.]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 4.216474597239768
fattori curr fine
curr 17.778658029168263
curr_answ 0
curr_answ 0
fattori curr inizio
curr_y 1
np.array(theta_stimato[curr_y]) [ 3.26752625e-04 -4.89475433e-01  2.45162495e+00]
np.array(Th[i].vector) [0.91 0.91 0.91]
W [[0.0, 0.0, 1.0], [0.0, 0.0, 1.0]]
d

np.array(Th[i].vector) [1.2 1.  1. ]
W [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
design_matrix(W[curr_y]) [[ 2.          5.          2.        ]
 [ 5.         12.66666667  4.66666667]
 [ 2.          4.66666667  3.66666667]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 5.695197871004753
fattori curr fine
curr 32.43527878989707
curr_answ 0
curr_answ 0
fattori curr inizio
curr_y 0
np.array(theta_stimato[curr_y]) [ 3.32143441 -1.78492352  2.19968143]
np.array(Th[i].vector) [0.91 0.91 0.91]
W [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
design_matrix(W[curr_y]) [[ 2.          5.          2.        ]
 [ 5.         12.66666667  4.66666667]
 [ 2.          4.66666667  3.66666667]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 4.973586540

norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 5.695197871004753
fattori curr fine
curr 32.43527878989707
curr_answ 0
curr_answ 0
fattori curr inizio
curr_y 0
np.array(theta_stimato[curr_y]) [ 3.32143441 -1.78492352  2.19968143]
np.array(Th[i].vector) [0.91 0.91 0.91]
W [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
design_matrix(W[curr_y]) [[ 2.          5.          2.        ]
 [ 5.         12.66666667  4.66666667]
 [ 2.          4.66666667  3.66666667]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 4.973586540407211
fattori curr fine
curr 24.736563074919765
theta_adv 5
best response 5
num_theta 6
curr_answ 0
fattori curr inizio
curr_y 0
np.array(theta_stimato[curr_y]) [ 3.32143441 -1.78492352  2.19968143]
np.array(Th[i].vector) [1.3 1.4 1.1]
W [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.33

...

q1
num_theta 6
curr_answ 1
fattori curr inizio
curr_y 1
np.array(theta_stimato[curr_y]) [19.98386008 -6.39142453 -0.32280297]
np.array(Th[i].vector) [1.3 1.4 1.1]
W [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
design_matrix(W[curr_y]) [[ 2.          5.          2.        ]
 [ 5.         12.66666667  4.66666667]
 [ 2.          4.66666667  3.66666667]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) 3.9919759313573895
fattori curr fine
curr 15.935871836536696
curr_answ 1
fattori curr inizio
curr_y 1
np.array(theta_stimato[curr_y]) [19.98386008 -6.39142453 -0.32280297]
np.array(Th[i].vector) [1. 1. 1.]
W [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
design_matrix(W[curr_y]) [[ 2.          5.          2.        ]
 [ 5.         12.66666667  4.66666667]
 [ 2.          4.66666667  3.6666666

W [[nan, nan, nan], [nan, nan, nan]]
design_matrix(W[curr_y]) [[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) nan
fattori curr fine
curr nan
curr_answ 1
curr_answ 1
fattori curr inizio
curr_y 0
np.array(theta_stimato[curr_y]) [18.20808263 -5.45925254 -1.38800942]
np.array(Th[i].vector) [0.91 0.91 0.91]
W [[nan, nan, nan], [nan, nan, nan]]
design_matrix(W[curr_y]) [[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) nan
fattori curr fine
curr nan
theta_adv -11
num_theta 6
curr_answ 1
fattori curr inizio
curr_y 0
np.array(theta_stimato[curr_y]) [18.20808263 -5.45925254 -1.38800942]
np.array(Th[i].vector) [1.3 1.4 1.1]
W [[nan, nan, nan], [nan, nan, nan]]
design_matrix(W[curr_y]) [[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[cu

C:\Users\edoar\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
C:\Users\edoar\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  



curr_answ 1
fattori curr inizio
curr_y 0
np.array(theta_stimato[curr_y]) [18.20808263 -5.45925254 -1.38800942]
np.array(Th[i].vector) [1. 1. 1.]
W [[nan, nan, nan], [nan, nan, nan]]
design_matrix(W[curr_y]) [[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) nan
fattori curr fine
curr nan
curr_answ 1
fattori curr inizio
curr_y 0
np.array(theta_stimato[curr_y]) [18.20808263 -5.45925254 -1.38800942]
np.array(Th[i].vector) [1.6 1.4 0.7]
W [[nan, nan, nan], [nan, nan, nan]]
design_matrix(W[curr_y]) [[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
norm_2_mat(np.array(theta_stimato[curr_y]) - np.array(Th[i].vector), design_matrix(W[curr_y])) nan
fattori curr fine
curr nan
curr_answ 1
fattori curr inizio
curr_y 0
np.array(theta_stimato[curr_y]) [18.20808263 -5.45925254 -1.38800942]
np.array(Th[i].vector) [1.2 1.  1. ]
W [[nan, nan, nan], [nan, nan, nan]]
design_matrix(W[curr_y]) [[nan nan nan]
 [nan nan nan

IndexError: list index out of range

In [11]:
print(theta_stimato)
for i in range (0, num_arms_y):
    print(np.dot(M[M_vero].matrix, np.transpose(A_y[i].arm_vector)))


print(W)
print(N_y)
print(N)
print(L_y)

[array([18.20808263, -5.45925254, -1.38800942]), array([19.98386008, -6.39142453, -0.32280297])]
[1.3 1.4 1.1]
[1. 1. 1.]
[[nan, nan, nan], [nan, nan, nan]]
[2, 3]
[[2, 1, 2], [2, 1, 2]]
[0.4348439081148646, 0.46645371317904005]


In [12]:
print("theta_stimato", theta_stimato)
print("A", A)
print("A_y", A_y)
print("num_arms", num_arms)
print("num_arms_y", num_arms_y)
print("num_theta", num_theta)
print("num_M", num_M)
print("M", M)
print("Th", Th)
print("W", W)
print("W_y", W_y)
print("N", N)
print("N_y", N_y)
print("Y", Y)
print("Y_y", Y_y)
print("H", H)
print("H_y", H_y)
print("L", L)
print("L_y", L_y)
print("Mixability_Gap", Mixability_Gap)
print("Mixability_Gap_y", Mixability_Gap_y)
print("AI", AI)
print("AI_y", AI_y)
print("nu", nu)
print("nu_y", nu_y)
print("eta", eta)
print("eta_y", eta_y)
print("M_vero", M_vero)
print("curr_answ", curr_answ)
print("curr_y", curr_y)

theta_stimato [array([18.20808263, -5.45925254, -1.38800942]), array([19.98386008, -6.39142453, -0.32280297])]
A [<__main__.arm object at 0x000001C5F0DCF3C8>, <__main__.arm object at 0x000001C5F0DCF548>, <__main__.arm object at 0x000001C5F0DCF648>]
A_y [<__main__.arm_y object at 0x000001C5F0DCF2C8>, <__main__.arm_y object at 0x000001C5F0DCF608>]
num_arms 3
num_arms_y 2
num_theta 6
num_M 3
M [<__main__.world object at 0x000001C5F0DCF348>, <__main__.world object at 0x000001C5F0C1EF88>, <__main__.world object at 0x000001C5F0C1E548>]
Th [<__main__.theta object at 0x000001C5F0E3F588>, <__main__.theta object at 0x000001C5F0DC5648>, <__main__.theta object at 0x000001C5F0E3F948>, <__main__.theta object at 0x000001C5F0E3FA88>, <__main__.theta object at 0x000001C5F0E3F9C8>, <__main__.theta object at 0x000001C5F0E3FB48>]
W [[nan, nan, nan], [nan, nan, nan]]
W_y [0.5007902444686076, 0.49920975553139246]
N [[2, 1, 2], [2, 1, 2]]
N_y [2, 3]
Y [[0, 6.386056580331619, 8.197770099053855, 4.130596824507

In [14]:
b.pull_arm()
c.pull_arm()
b.pull_arm()

10.491507210938886

In [15]:
Y

[[0,
  6.386056580331619,
  8.197770099053855,
  4.130596824507053,
  6.960539856045385,
  6.368249109314574,
  12.245837981676537,
  6.388602902585354,
  10.468931085241058,
  11.778150175828676,
  7.050542350227168,
  10.491507210938886],
 [0,
  6.386056580331619,
  8.197770099053855,
  4.130596824507053,
  6.960539856045385,
  6.368249109314574,
  12.245837981676537,
  6.388602902585354,
  10.468931085241058,
  11.778150175828676,
  7.050542350227168,
  10.491507210938886]]

In [16]:
N

[[2, 5, 4], [2, 5, 4]]

In [17]:
W = [1.1, 2.6, 1.3]

TypeError: best_response() takes 0 positional arguments but 1 was given

In [ ]:
best_response(0)

In [21]:
np.array(theta_stimato) - np.array(Th[best_response(0)].vector)

TypeError: best_response() takes 0 positional arguments but 1 was given

In [ ]:
np.array(A[0].arm_vector)

In [ ]:
p1 = np.matmul((np.array(theta_stimato) - np.array(Th[best_response(0)].vector)), np.array(A[0].arm_vector))
p1

In [ ]:
math.sqrt(2*beta(t, 1/t**3))

In [ ]:
np.sqrt(2*beta(t, 1/t**3))

In [ ]:
k = 1000
time = k
delta = 1/k**3


In [ ]:
n = np.abs(np.random.normal(0, 1))
n

In [ ]:
math.sqrt(math.log10(1/delta))

In [ ]:
d/2*math.log10(1 + (time*(l**2))/(n*d))

In [ ]:
math.sqrt(n/2)*m

In [ ]:
beta(k, 1/k**3)

In [ ]:
p2 = np.sqrt(2*beta(t, 1/t**3))*norm_2_mat(np.array(A[0].arm_vector), design_matrix(N) + np.random.normal(0, 1)*np.identity(d))

In [ ]:
calcola_U(0, 2)

In [ ]:
calcola_U(1, 2)

In [ ]:
calcola_U(2, 2)

In [ ]:
for i in range (0 , T//3 - t):
    a.pull_arm()
    b.pull_arm()
    c.pull_arm()
    t += 3

In [ ]:

stima_theta()

In [ ]:
for i in range (0, num_arms):
    print(best_response(i))

In [ ]:
for i in range (0, num_arms):
    print(np.matmul(A[i].arm_vector, Th[theta_vero].vector))

In [ ]:
for i in range (0, num_arms):
    print(calcola_U(i, best_response(i))/100000000)

In [ ]:
S = [[]]
S.append([])
S.append([])
S

In [ ]:
S[0] = [0]*3
S

In [ ]:
W.append([])
N.append([])
L.append([])
for i in range (0, num_arms):
    W[id].append(0)
    N[id].append(0)
    L[id].append(0)

In [ ]:
S

In [ ]:
S[0].append(1)
S[0].append(2)